# Load Data

In [3]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F 

In [10]:
X_train = pd.read_csv('train.csv')
X_test = pd.read_csv('test.csv')

In [40]:
X = torch.tensor(X_train.iloc[:, 1:].values).float()

In [41]:
X.shape # (example, image size)

torch.Size([42000, 784])

In [42]:
y = torch.tensor(X_train.iloc[:, :1].values)

In [43]:
y.shape

torch.Size([42000, 1])

In [44]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(784, 200)
        self.l2 = nn.Linear(200, 10)
    def forward(self, x):
        x = self.l1(x)
        x = self.l2(F.relu(x))
        return x

In [ ]:
myyNet = Net()
logits = myyNet(X)
logits.shape

# 64, 10
# 64, 1

torch.Size([42000, 10])

In [51]:
myNet = Net()
optimizer = torch.optim.Adam(myNet.parameters(), lr=0.01)

for i in range(1000):
    optimizer.zero_grad()
    ix = torch.randint(0, X.shape[0], (64,))
    logits = myNet(X[ix])
    loss = F.cross_entropy(logits, y[ix].squeeze())
    loss.backward()
    optimizer.step()
    print(loss.item())

30.319316864013672
190.01792907714844
272.54302978515625
271.14404296875
152.07479858398438
113.38319396972656
72.0702896118164
37.94154357910156
34.09791564941406
27.18926239013672
23.10432243347168
11.194985389709473
12.3928804397583
10.573169708251953
7.829804420471191
8.449982643127441
5.517839431762695
3.6237871646881104
3.9610228538513184
5.5176897048950195
3.707852602005005
3.532510280609131
2.5665595531463623
3.082792043685913
2.0204389095306396
1.288165807723999
2.6221137046813965
1.5437183380126953
0.985933780670166
1.808074951171875
1.634605050086975
3.036262035369873
1.4073361158370972
1.2971255779266357
2.1309797763824463
1.3652708530426025
2.604405403137207
1.1874943971633911
1.4712135791778564
2.153951644897461
1.771677851676941
1.5592713356018066
1.169910192489624
2.246293306350708
1.16709566116333
1.3561291694641113
1.5144375562667847
1.2770135402679443
1.541141390800476
1.494231939315796
1.1393351554870605
1.334092617034912
1.1671887636184692
1.3434334993362427
1.5343